# Task 6 

Subtasks:

a) Create a server-side deactivatable system to pseudonymize data;

b) Create a dictionary of recurrent single words and n-grams of two, three or more words (typical sentences used) from governmental databases on sexual harassment, newspaper and laws about the issues, better from Nigeria;

c)  Classify sentences;

d) Create a server-side deactivatable system to allow human control.


## Subtask b) -- Create a dictionary of rilevant single words and n-grams of two, three or more words (typical sentences used) from datasets or corpora

In [ ]:
!pip list

In [1]:
import pandas as pd
import ast
import seaborn as sns 
import numpy as np
import pandas as pd
import matplotlib
import re
import sys

#import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from collections import Counter
from datetime import datetime

%matplotlib inline

## Functions

###  Dictionaries

In [2]:
#open dictionary

def openDictionary(file):
    f = open(file,'r')
    text = f.read()       
    dictio = ast.literal_eval(text)
    f.close()
    #print('length =',len(dictio))
    #print(type(dictio))
    #print(dictio)
    return dictio

### Check for unbalanced parentheses in dictiory

In [3]:
# Python3 code to Check for 
# balanced parentheses in an expression
def check(my_string):

    
    
    
    brackets = ['()', '{}', '[]']
    while any(x in my_string for x in brackets):
        for br in brackets:
            my_string = my_string.replace(br, '')
    return not my_string



# Driver code

f = open('./dictionaries/female.txt','r')
text = f.readlines()

for line in text:
    print(line)
    pattern = r"[a-z]|[-.*?\s\\':,|]|[1-9]"
    strg = re.sub(pattern, '', line )
    print(strg)
    print(strg, "-", "Balanced" 
      if check(strg) else "Unbalanced")
    print('...............')




FileNotFoundError: [Errno 2] No such file or directory: './dictionaries/female.txt'

### Priotizing by source


In [ ]:
prior_source = {
    'youtube':3,
    'facebook':2,
    'tweeter':2,
    'news':3,
    'linkedin':3,
    'stories':2,
    'help_center':1
}


### Dataframe

In [ ]:
#set the dataframe by file name, column and source
def openDf(file_name, column, source, source_priority, homogeneity):
    
    #open the file 
    datafile = pd.read_csv(file_name)
    
    #select the column with text
    df = datafile.loc[:,[column]]
    
    #rename column with text
    df.rename(columns = {column :'body'}, inplace = True)
    
    # take into account the source and homogeneity
    df['source'] = ""
    df['source_priority'] = ""
    df['homogeneity'] = ""
    
    #row by row, source and source_priority declaration 
    for index, row in df.iterrows():
        row['source'] = source
        row['source_priority'] = source_priority
        row['homogeneity'] = homogeneity

        
    #move columns
    df = df[['homogeneity', 'source', 'source_priority','body']]
    
    # create columns to manage tags and weights
    df['tags_a'] = ""
    df['sum_wgts_a'] = ""
    df['tags_b'] = ""
    df['sum_wgts_b'] = ""    
    df['tags_c'] = ""
    df['sum_wgts_c'] = ""  
    df['tags_other'] = "" 


    # create columns to manage joint tags and joint weights
    df['tags_ab'] = ""
    df['sum_wgts_ab'] = ""
    df['tags_bc'] = ""
    df['sum_wgts_bc'] = ""
    
    # create columns to in & out  
    df['tags_f'] = "" 
    df['inout'] = ""
    
    # create columns to prioritize by source and by violence's type
    df['violence_type'] = ""

    df['priority'] = ""    
       
    #sum tags and weights of ab,bc and c
    df['tot_tags'] = ""
    df['tot_wgts'] = ""
    
    #lenghts of text criteria
    df['text_len_criteria'] = ""
    
    # create column to manage validation, reports and delete columns
    df['val'] = ""
    return df

### Tagging

In [ ]:
# create tags from dictionary
def tagging(vocab):
    
    tags = []
    weights = []

    # search amongs fundWords
    for key,value in vocab.items():
        
        # if terms in row
        if re.findall(key, row.body):
            
            #append tags
            tags.append(value[1])
            #append weights
            weights.append(value[0])
            
    #delete duplicate tags    
    tags_cleaned = list(dict.fromkeys(tags))
    
    #sum weights      
    weights_summed = sum(weights)
    
    #return list of tags and weights
    return tags_cleaned, weights_summed
    
    
    

In [ ]:
def tagging_other(vocab):
    
    tags = []


    # search amongs fundWords
    for key in vocab:
        
        # if terms in row
        if re.findall(key, row.body):
            
            #append tags
            tags.append(key)
            
    #delete duplicate tags    
    tags_cleaned = list(dict.fromkeys(tags))
    
    #return list of tags and weights
    return tags_cleaned

In [ ]:
def tagging_c(vocab):
  
    #prepare lists
    tags = []
    weights = []
    
    #word frequency
    word_counter = Counter(row['body'].split())
    
    #word by word 
    for w, v in word_counter.items():
        
        #terms by terms in dictionary 3 
        for i in vocab:
            
            #if terms in row
            if re.findall(i, w):
                
                #append tags
                tags.append((w,v))
                #append weights
                weights.append(v)
    
    #sort tags in base to its value 
    from operator import itemgetter, attrgetter    
    tags_sorted = sorted(tags, key=itemgetter(1), reverse=True)

    #sum weights      
    weights_summed = sum(weights)
    
    return tags_sorted, weights_summed



### Sorting

In [ ]:
#sort dataframe
def sort_by(column, asc):
    df.sort_values(by=column, ascending=True, inplace=True)

    

### Dropping rows

In [ ]:
#drop row by index
def drop_row(index_list):
    print('are you sure of deleting the following row?', index_list)
    x = input()
    if x == 'y':
        df.drop(index_list, inplace=True)
        print('deleted', index_list)
    else: sys.exit()
    


### Reporting

In [ ]:
# save report by name
def df_report(report_name, df_name):
    with open(report_name, 'w') as f:
        for index, row in df_name.iterrows():
            f.write('\n**********************************************\n')
            f.write(str(index))
            f.write('-|-')
            f.write(row.body)
            f.write('\n')
            f.write('----------IN & OUT------------\n')
            f.write(' '.join([str(elem) for elem in row.inout]))
            f.write('\n')
            f.write('----------tags_a------------\n')
            f.write(' '.join([str(elem) for elem in row.tags_a]))
            f.write('\n')
            f.write('----------tags_b------------\n')
            f.write(' '.join([str(elem) for elem in row.tags_b]))        
            f.write('\n')
            f.write('----------tags_f------------\n')
            f.write(' '.join([str(elem) for elem in row.tags_f]))        
            f.write('\n')
            f.write('----------tags_c------------\n')
            f.write(' '.join([str(elem) for elem in row.tags_c]))        
            f.write('\n')
            f.write('----------tags_other------------\n')
            f.write(' '.join([str(elem) for elem in row.tags_other]))        
            f.write('\n')            
            #f.write('----------tags_ab------------\n')
            #f.write(' '.join([str(elem) for elem in row.tags_ab]))  
            #f.write('\n')
            #f.write('----------tags_bc------------\n')
            #f.write(' '.join([str(elem) for elem in row.tags_bc])) 
            #f.write('\n')
            f.write('----------tot_tags------------\n')
            f.write(' '.join([str(elem) for elem in row.tot_tags]))
            f.write('\n')
            f.write('----------tot_wgts------------\n')
            f.write(str(row.tot_wgts))

    
    
    


### Human_control

In [ ]:
# control the file by name, row index and drop rows by val
def human_control(df_name, row_index):
    
    #open the file 
    df = pd.read_csv(df_name)
    
    # set starting index
    row_index = int
    
    list_index = []

    #row by row
    for index, row in df.iterrows():
        
        print('\n**********************************************\n')
        
        print(row_index)
        print(row.body)
        print('\n')
        print('----------IN & OUT------------\n')
        print(' '.join([elem for elem in row.inout]))
        print('\n')
        print('----------tags_a------------\n')
        print(' '.join([elem for elem in row.tags_a]))
        print('\n')
        print('----------tags_b------------\n')
        print(' '.join([str(elem) for elem in row.tags_b]))        
        print('\n')
        print('----------tags_f------------\n')
        print(' '.join([str(elem) for elem in row.tags_f]))   
        print('\n')
        print('----------tags_c------------\n')
        print(' '.join([str(elem) for elem in row.tags_c]))   
        print('\n')
        print('----------tags_other------------\n')
        print(' '.join([str(elem) for elem in row.tags_other]))        
        print('\n')
        #print('----------tags_ab------------\n')
        #print(' '.join([str(elem) for elem in row.tags_ab]))  
        #print('\n')
        #print('----------tags_bc------------\n')
        #print(' '.join([str(elem) for elem in row.tags_bc])) 
        #print('\n')
        #print('----------tot_tags------------\n')
        #print(' '.join([str(elem) for elem in row.tot_tags]))
        #print('\n')
        print('----------tot_wgts------------\n')
        print(row.tot_wgts)
        print('\n')
        print(row.index)        
        print('\n')
        print('row in the list for changes or deletions?')
        print('y/n or exit with any other character')
        x = input()
        
        if x == 'y' or x == 'n':
            row['val'] = x
            if x == 'y':
                list_index.append('row_index')            
        else:
            sys.exit()
            
    print(list_index)
    return list_index
    

### Saving

In [ ]:
#save the dataframe with name
def save_df(name):   
    
    df.to_csv(name)
    return df.head()

# -------------------------------------------------------------------------------------------------------

## Execution

## problem: homogeneous (dataset is only about women and violence against them) / inhomogeneous dataset is about women, men, lgbt groups, mental illnesses, economy, finance, politics)


## solution: two paths, one for homogeneous dataset, one for inhomogeneous dataset (we need to understand if the story is about violence against women) 


## problem: different sources (facebook, tweeter, youtube, etc...) can have affect about priorities 

## solution: to prioritize sources


## problem: the fundamental terms are few, understanding the context is difficult even for a human

## solution: build dictionaries using regular expression massively



## I.E. - words used to identify abuse and genital mutilations :

## '(abus(e|ed|es|ing|ive|ives|ived) (wo(man|men)|gir(l|ls)|femal(e|es)|girlfrien(d|ds)|lad(y|ies)|her))':(10, 'in general, abuses'),

## '(abus(e|ed|es|ing|ive|ives|ived) against (wo(man|men)|gir(l|ls)|femal(e|es)|girlfrien(d|ds)|wif(e|es)|spous(e|es|al)|lad(y|ies)|her))':(10, 'in general, abuses'),

## '((wo(man|men)|gir(l|ls)|femal(e|es)|girlfrien(d|ds)|lad(y|ies)|her) abus.*?\s)':(10, 'in general, abuses'),

## '((spou(se|ses|sal|sals)|wif(e|es)|married) abus.*?\s)':(10, 'spousal abuse'),

## '(circumcis(ed|ing|e|es|ion|ions) (wo(man|men)|gir(l|ls)|femal(e|es)|girlfrien(d|ds)|wif(e|es)|spous(e|es|al)|lad(y|ies)|her))':(10, 'genital mutilation'),

## '((wo(man|men)|gir(l|ls)|femal(e|es)|girlfrien(d|ds)|wif(e|es)|spous(e|es|al)|lad(y|ies)|her) circumcis.*?\s)':(10, 'genital mutilation'),

## '((wo(man|men)|gir(l|ls)|femal(e|es)|girlfrien(d|ds)|wif(e|es)|spous(e|es|al)|lad(y|ies)|her) genital mutilat.*?\s)':(10, 'genital mutilation'),

## '((wo(man|men)|gir(l|ls)|femal(e|es)|girlfrien(d|ds)|wif(e|es)|spous(e|es|al)|lad(y|ies)|her) genital cut.*?\s)':(10, 'genital mutilation'),


### Dictionaries

In [ ]:
# Vocabulary of terms that identify violence against women in combination with other terms
# i.e.
fundWords = openDictionary('./dictionaries/fundamentalWords.txt')

# Vocabulary of terms that might identify violence against women in combination with other terms
#i.e abusive marriage with sex, sexual, she, girl, girls, woman, women,lady, ladies, violence
impWords = openDictionary('./dictionaries/importantWords.txt')

#female terms 
female = openDictionary('./dictionaries/female.txt')


#other terms (potremmo inserire posti geografici o malattie come hiv, aids, etc...)
word_list = openDictionary('./dictionaries/wordList.txt')

#other terms (potremmo inserire posti geografici o malattie come hiv, aids, etc...)
other = openDictionary('./dictionaries/others.txt')

In [ ]:
# Vocabulary of terms that identify violence against women in combination with other terms
# i.e.
fundWords = openDictionary('./dictionaries/fundamentalWords.txt')

# Vocabulary of terms that might identify violence against women in combination with other terms
#i.e abusive marriage with sex, sexual, she, girl, girls, woman, women,lady, ladies, violence
impWords = openDictionary('./dictionaries/importantWords.txt')

#female terms 
female = openDictionary('./dictionaries/female.txt')


#other terms (potremmo inserire posti geografici o malattie come hiv, aids, etc...)
word_list = openDictionary('./dictionaries/wordList.txt')

#other terms (potremmo inserire posti geografici o malattie come hiv, aids, etc...)
other = openDictionary('./dictionaries/others.txt')

### Dataframe

## when we build the dataset we need to take into account the source and homogeneity

## to prioritize sources

In [ ]:
prior_source

## what part of the dataset contains the text to be analyzed?

In [ ]:
#set facebook dataset file name, column and source

dtfrm = pd.read_csv('youtube_transcripts(copia).csv')

print(list(dtfrm))
dtfrm.head()

In [ ]:
#set df by file name, column and source
df = openDf('youtube_transcripts(copia).csv', 'transcript', 'youtube transcripts', 3, 'inhomogeneous')
df.head()

In [ ]:
print(list(df))

### Tagging

## problem: having an overall view of what the text is about 

## solution: tagging using individual dictionaries

## - tags_a tells us if a text deals with violence against women for sure.
## - tags_b identifies forms of violence that can be against women.
## - tags_f is used to confirm that the violence of tag_b can be against women.
## - tags_c gives an overall view of what the text is about 
## - tags_other can be used to eventually cross-reference data. For example, we might be interested in cross-referencing domestic violence data with particular regions of nigeria or with covid. The 'other.text' dictionary might contain words about regions of nigeria or covid.

## problems: to prioritize records with more information.

## solution: to use word values in the dictionary to give a weight to that word and then sort data by highest score


In [ ]:
## Preprocessing Text

#clean text from emojoy
#print(df['body'].iloc[1])
df['body'] = df['body'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)
#print('---------------')
#print(df['body'].iloc[1])

# delete nan row
df.dropna(subset = ["body"], inplace=True)

# delete duplicates
df.drop_duplicates(subset=['body'], inplace=True)

#ignore case
df['body'] = df['body'].str.lower()


#print(df['body'].iloc[1])

In [ ]:
### tags_a tells us if a text deals with violence against women for sure.
### tags_b identifies forms of violence that can be against women.
### tags_c is used to confirm that the violence of tag_b can be against women.
 

#row by row
for index, row in df.iterrows():
    
   
    #create and insert tags in tags_a
    row['tags_a'] = tagging(fundWords)[0]    
    # sum_wgts_a assigns a weight to tags_a so that the higher the score, the more important the story 
    row['sum_wgts_a'] =  tagging(fundWords)[1]
    # to join tags_a and tags_b we create and insert tags_a in tags_ab 
    row['tags_ab'] = tagging(fundWords)[0]
    
    #create and insert tags in tags_b    
    row['tags_b'] = tagging(impWords)[0]
    # to join tags_b and tags_c we create and insert tags_b in tags_bc 
    row['tags_bc'] = tagging(impWords)[0]
    ## sum_wgts_b assigns a weight to tags_b so that the higher the score, the more important the story
    row['sum_wgts_b'] =  tagging(impWords)[1]
    
    #create and insert tags in tags_f
    row['tags_f'] = tagging_other(female)
    
    #create and insert tags in tags_f
    row['tags_other'] = tagging_other(other)
    
    #create, sort, insert tags in tags_c 
    from operator import itemgetter, attrgetter    
    row['tags_c'] = tagging_c(word_list)[0]
    row['sum_wgts_c'] = tagging_c(word_list)[1]


    

    

In [ ]:
### Not being sure that the text contains violence against women, we joint tags_b and tags_a 
### and then tags_b and tags_c. 
### Tags_ab confirm for sure violence against women 
### Tags_bc helps understand if the story is related to women

#row by row
for index, row in df.iterrows():

    # if row has tags_a 
    if len(row['tags_ab']) != 0:
        #merging tags_a and tags_b in 
        row['tags_ab'] =  list(dict.fromkeys(row['tags_ab']+row['tags_b']))  
    #sum weights A with weights B 
    row['sum_wgts_ab'] = row['sum_wgts_a'] + row['sum_wgts_b']
    
    # if row has tags_b 
    if len(row['tags_bc']) != 0:
        #merging tags_b and tags_c in 
        row['tags_bc'] =  list(dict.fromkeys(row['tags_bc']+row['tags_c']))  
    #sum weights A with weights B 
    row['sum_wgts_bc'] = row['sum_wgts_b'] + row['sum_wgts_c']
    
    #all together
    #row['tot_tags'] = list(dict.fromkeys(row['tags_ab']+row['tags_bc']+row['tags_c']))  
    row['tot_tags'] = list(dict.fromkeys(row['tags_a']+row['tags_b']+row['tags_c']))
    
    #sum weights A with weights B 
    cou = len(Counter(row['body'].split()))
    row['tot_wgts'] = row['sum_wgts_a'] + row['sum_wgts_b'] + row['sum_wgts_c']
    
    #by text length criteria
    #row['text_len_criteria'] = (row['tot_wgts']/cou)

    


### In and out

## problem: to sort records by whether they relate to violence against women or otherwise

## solution: 

## - A identifies records that are definitely violence against women
## - B identifies records that are almost certainly violence against women
## - C identifies records that are perhaps violence against women
## - D identifies records that does not seem violence against wome
## - E identifies records that are not violence against women


In [ ]:
for index, row in df.iterrows():
    
    #A(definitely violence against women)
    if len(row['tags_a']) != 0:
        row['inout'] = 'A = definitely violence against women'
 
    #B(almost certainly violence against women)
    elif len(row['tags_a']) == 0 and len(row['tags_b']) !=0 and len(row['tags_f']) != 0:
        row['inout'] = 'B = almost certainly violence against women'

    #C(perhaps violence against women)
    elif len(row['tags_a']) == 0 and len(row['tags_b']) != 0 and len(row['tags_f']) == 0 and len(row['tags_c']) != 0:
        row['inout'] = 'C = perhaps violence against women' 

    #D(it does not seem violence against women)
    elif len(row['tags_a']) == 0 and len(row['tags_b']) == 0 and len(row['tags_f']) != 0 and len(row['tags_c']) != 0:
        row['inout'] = 'D = it does not seem violence against women'     

    #should not be violence against women
    else: row['inout'] = 'E = it is not violence against women' 

### In & Out -- new dataframe of selected columns: sort, save and report 

## problem: how to enable human control

## solution: 
## 1. reports
## 2. realtime control 

## 1. reports

In [ ]:
#New dataframe
df1 = df[['body','inout','tags_a','tags_b','tags_f','tags_c','tags_other','tot_tags','tot_wgts']]

#sort values
df1.sort_values(['inout','tot_wgts'], ascending=[True, False], inplace=True)
df1_n = df1.to_csv('YT_inout.csv')

#report by name

name = ('YT_REPORT_inout_'+datetime.now().strftime('_%d-%m-%d_%H-%M-%S')+'.txt')
print(name)
rep = df_report(name, df1)



## 2. realtime control 

In [ ]:
#In & Out -- human_control
human_control('YT_inout.csv', 0)

### In & Out -- dropping columns

## problem: delete records not relevant to our purpose

## solution: drop the records according to the report index

In [ ]:

ind_lst = []
drop_row(ind_lst)


### Saving

In [ ]:
#save the cleaned dataframe
f_name = ('YT_priotized_cleaned_'+datetime.now().strftime('_%d-%m-%d_%H-%M-%S')+'.csv')
print(f_name)
save_df(f_name)

### Querying

## at this point we have a dataset of informative records that we can query and with which it is possible to filter and cross data and make reports 

In [ ]:
df['tot_tags']

In [ ]:
df['tot_string_tags'] = [','.join(map(str, l)) for l in df['tot_tags']]
df['tot_string_tags']

In [ ]:
dom_viol = df[df['tot_string_tags'].str.contains('domestic violence') & df['tot_string_tags'].str.contains('help')]
dom_viol1 = df[df['tot_string_tags'].str.contains('domestic violence')]

In [ ]:
print(df.shape, dom_viol.shape, dom_viol1.shape)
print()
print('---------')
print(df.info())
print()
print('---------')
print(dom_viol.info())
print()
print('---------')
print(dom_viol1.info())

### Sorting

In [ ]:
#sorted dataframe by ...
#sort_by('in&out')
#sort_by('priority')
#sort_by('sum_wgts_a')
#sort_by('sum_wgts_b')
#sort_by('sum_wgts_c')
#sort_by('tot_wgts')
#sort_by('text_len_criteria')

#df.sort_values(['priority','tot_wgts'], ascending=[True, False], inplace=True)


### Sorting  and reporting

In [ ]:
# create report by name
df.sort_values(['priority','tot_wgts'], ascending=[True, False], inplace=True)
df_report('YT_REPORT_5.txt')

### Saving

In [ ]:
#save the cleaned dataframe
save_df('FB_cleaned_priotized_transcripts.csv')
#save_df('youtube_cleaned_transcripts1.csv')
#save_df('youtube_cleaned_transcripts2.csv')



### Human Control

In [ ]:
#human_control(youtube_cleaned_transcripts.csv, 0)

# Results

## we have a dataset of informative records that we can query and with which it is possible to filter and cross data and make reports

## we have created a dictionary of relevant single words and n-grams of two, three or more words (typical sentences used) from databasets or corpora

## we have a server-side system to allow human control

## This tool is already able to identify and prioritize forms of violence against women


# NEXT STEPS

## within next week:

## - to improve weights by grouping tags

## - to make a single dataset of selected records 

## - to try a supervised classification method

## within two weeks:

## - to check results, refine and put all together




   


Correctives:
- using verbs
- relationships with geography and diseases (hiv, aids, etc.. )
- improve weights by grouping tags
